In [1]:

%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint
from joblib import dump, load

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd
from multiprocessing import Pool

import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from rapidfuzz import process, fuzz
from sklearn.neural_network import MLPClassifier

from difflib import SequenceMatcher
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'

r = redis.Redis(host="127.0.0.1", port=6379, db=0)


from sqlalchemy import event,create_engine,types
driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

In [2]:
li_all_provider = list(pd.read_csv('./gibberish-detector/all_email_provider_domains.txt',
                 names=['all_provider'])['all_provider'])

def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(email_input):

    
    name_input = email_input.split('@')[0]
    provider_input = email_input.split('@')[-1]
    
   
    pat_count_digit = encode(name_input).count('D')
    pat_encode = short_encode(name_input)
   
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 1
    elif pat_encode == 'D':
        return 1
    elif provider_input not in li_all_provider:
        return 1
    return 0
    
score_sus_email('akhiyar.waladi@hehehe.com')

1

In [3]:
from gibberish_detector import detector

path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model.test'
Detector = detector.create_from_model(
    path_detector_indo
)

def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    
    score_gib = Detector.calculate_probability_of_being_gibberish(split_email)
    
    return score_gib

email = 'fadmawdwa@gmail.com'
score_gibberish(email)



3.7002451204535642

In [ ]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member_01jan22.json',
                              dtype='object')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)
alfagift_member['created_at'] = pd.to_datetime(alfagift_member['created_at'])+ pd.DateOffset(hours=7)
alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")


alfagift_member['birth_date'] = alfagift_member['birth_date'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)
# alfagift_member['birth_date'] = pd.to_datetime(alfagift_member['birth_date']).dt.strftime("%Y-%m-%d")
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')



In [ ]:
# ### creating label for training set


# li_sus = []
# for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
#     sel_created = dr.strftime("%Y-%m-%d")

#     alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
#     #print(alfagift_member_sel_d.shape)


#     ## suspect by IP
#     alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
#                             .agg(createdFromIp_count=('createdFromIp','count'))
#     alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
#         alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

#     sus_by_ip = pd.merge(alfagift_member_sel_d_ip, alfagift_member_sel_d, on='createdFromIp', how='left')

#     ## suspect by phone
#     alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
#     alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
#                                 .agg(createdFromIp_count=('phone_sub','count'))
#     alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
#         alfagift_member_sel_d_phone['createdFromIp_count'] >= 5].reset_index()

#     sus_by_phone = pd.merge(alfagift_member_sel_d_phone, alfagift_member_sel_d, on='phone_sub', how='left')

    
#     alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    
    
#     ## saving by concating suspect aspect
#     sus_all = pd.concat([
#         sus_by_ip[['account_card','email','phone']], 
#         sus_by_phone[['account_card','email','phone']],
#         alfagift_member_sel_d[alfagift_member_sel_d['member_status'] == 27][['account_card','email']],
#         alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] > 4][['account_card','email']]
#     ], axis=0)\
#         .drop_duplicates()
    
#     sus_all = sus_all[~sus_all['email'].str.contains('deliverysapa', case=False)]
#     sus_all = sus_all[['account_card','phone','email']]
    
    
#     ## inserting into table to check transaction
#     df = sus_all[['account_card']]

#     engine = create_engine(engine_stmt)


#     dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
#     df.to_sql('temp_sus_member', engine, index=False, 
#                                           if_exists="replace", dtype=dtyp)
    
    
#     q = '''
    
#         SELECT 
#             rto.TRO_DATE, rto.TRO_PONTA_ID
#         FROM 
#             temp_sus_member tsm
#             INNER JOIN RPT_TRANS_OSHOP rto
#             ON rto.TRO_PONTA_ID = tsm.ACCOUNT_CARD 
    
#     '''
#     con_alfabi = engine.connect()
#     df_exclude = pd.read_sql_query(q, con_alfabi)
#     df_exclude_g =  df_exclude.groupby('tro_ponta_id').agg(tro_date_count=('tro_date','nunique'))
#     df_exclude_g = df_exclude_g[df_exclude_g['tro_date_count'] >2].reset_index()

    
#     con_alfabi.close()
    
#     sus_all = sus_all[~sus_all['account_card'].isin(df_exclude_g['tro_ponta_id'])]
#     ### end inserting into table to check transaction
#     li_sus.append(sus_all)


#     #break
#     sus_all.to_csv(
#         '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created)
#     )

    

# df_sus_all = pd.concat(li_sus).drop_duplicates()
# df_sus_all.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_all_exclude_trx.csv', index=False)

In [ ]:

def seq_match(string1, li_string):

    
    match_start = 0
    match_end = 0
    for string2 in li_string:
        match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

        
        if match.size >= 3:


            if ( string1.startswith(string1[match.a:match.a + match.size]) 
                and  string2.startswith(string2[match.b:match.b + match.size]) ) :
                
                match_start += 1
                
            if ( string1.endswith(string1[match.a:match.a + match.size]) 
                and string2.endswith(string2[match.b:match.b + match.size]) ):
                
                match_end += 1
        
        
        
    return match_start, match_end
    


In [ ]:
# ## creating feature for training set


# li_sus = []
# for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
#     sel_created = dr.strftime("%Y-%m-%d")

#     alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
#     # print("{} {}".format(sel_created, alfagift_member_sel_d.shape))
#     alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
#     alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()
    
#     ## suspect by IP
#     alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
#                             .agg(createdFromIp_count=('createdFromIp','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_ip,
#                                      on='createdFromIp', how='left').fillna(0)

#     ## suspect by phone
#     alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
#                                 .agg(phoneSub_count=('phone_sub','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_phone,
#                                      on='phone_sub', how='left').fillna(0)
    
    
#     alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
#     alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    
# #     alfagift_member_sel_d['nameMatch_count'] = alfagift_member_sel_d['full_name']\
# #     .apply(lambda x: len(process.extract(x, alfagift_member_sel_d['full_name'], 
# #             score_cutoff=70, limit=alfagift_member_sel_d.shape[0], scorer=fuzz.token_sort_ratio))-1)
 
#     alfagift_member_sel_d = parallelize_dataframe(alfagift_member_sel_d, add_features)


#     li_sus.append(alfagift_member_sel_d)
#     alfagift_member_sel_d.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))

#     break

# # df_sus_all = pd.concat(li_sus)
# # df_sus_all.to_csv(
# #     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
# #     , index=False)

In [ ]:
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

def square(dr):


    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]\
                            .iloc[:,2:].reset_index(drop=True)
    
    
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()

    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_ip]
                                     ,axis=1)

    ## suspect by phone
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(phoneSub_count=('phone_sub','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_phone]
                                     ,axis=1)
    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    

    li_name_count = []
    for idx, row in alfagift_member_sel_d.iterrows():
        cur_name = row['full_name']

        compare_name = list(alfagift_member_sel_d.iloc[0:idx+1,:]['full_name'])


        res_compare = seq_match(cur_name, compare_name)
        li_name_count.append(res_compare)


    df_count_name = pd.DataFrame(li_name_count, columns=['full_name_sCount', 'full_name_eCount'])
    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, df_count_name], axis=1)

    alfagift_member_sel_d.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily_gmt/{}'.format(sel_created)
    ,index=False)


    return 0


start = timer()

print(f'starting computations on {cpu_count()} cores')

values = pd.date_range('2021-12-01', '2022-04-20', freq='D')

with Pool(48) as pool:
    res = pool.map(square, values)
    

end = timer()
print(f'elapsed time: {end - start}')

In [ ]:
pd.date_range('2021-12-01', '2022-04-20', freq='D')